In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

js = "https://angel.co/company_filters/search_data"

headers = {"X-Requested-With": "XMLHttpRequest",
           "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36"}

u = "https://angel.co/companies/startups?ids%5B%5D={}&total={}&page={}&sort=signal&new=false&hexdigest={}"
with requests.Session() as s:
    params = s.post(js, data={"sort": "signal"}, headers=headers).json()
    companies = s.get(u.format("&ids%5B%5D=".join(map(str, params["ids"])),params["page"] ,params["total"], params["hexdigest"]), headers=headers)
    soup = BeautifulSoup(companies.json()["html"], "html.parser")
    
# with open("code.html", "w") as file:
#     file.write(str(soup))

companies = soup.findAll(name="div", attrs={"class": "base startup"})

In [2]:
result = []
for idx, company in enumerate(companies):
    if idx % 4 == 0:
        print('{} company'.format(idx))
    name = company.findAll("a", {"class": "startup-link"})[1].text

    description = company.findAll("div", {"class": "pitch"})[0].text.strip('\n')
    if len(description) == 0:
        description = '-'

    company_column = company.findAll("div", {"class": "company column"})[0]
    angelListUrl = company_column.findAll('a', href=True)[0]['href']

    location_tag = company.findAll("div", {"class":"location"})[0]
    location = location_tag.findAll("div", {"class":"value"})[0].text.strip('\n')
    
    employees_tag = company.findAll("div", {"class": "company_size"})[0]
    employees = employees_tag.findAll("div", {"class":"value"})[0].text.strip('\n')
    
    raised_tag = company.findAll("div", {"class": "raised"})[0]
    raised = raised_tag.findAll("div", {"class": "value"})[0].text.strip('\n')
    
    website_tag = company.findAll("div", {"class": "website"})[0]
    a = website_tag.findAll('a', href=True)
    website = a[0]['href']
    if len(website) == 0:
        website = '-'
    
    full_company = [name,description,angelListUrl,location,employees,raised,website]
    result.append(full_company)

0 company
4 company
8 company
12 company
16 company


In [3]:
df = pd.DataFrame(result, columns=['name','desc','angelListUrl','location','employees',
                                   'raised','website'])
df.head()

,name,desc,angelListUrl,location,employees,raised,website
0,Blissfully,"Blissfully automates SaaS tracking, savings, a...",https://angel.co/blissfully,Boston,1-10,"$1,500,000",http://www.blissfully.com/
1,SV Liquidity,Fund providing access to IPO opportunities,https://angel.co/hf-liquidity,San Francisco,1-10,-,https://svlq.io
2,Tetra (YC W17),AI notetaker for meetings,https://angel.co/tetra,San Francisco,1-10,-,https://asktetra.com
3,Zipline International,-,https://angel.co/zipline-international-1,-,-,"$25,000,000",-
4,Stypi,-,https://angel.co/stypi,-,,-,-


In [4]:
df.to_csv('parsed_20companies.csv')